In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Human input

In [2]:

# AI Researcher Agent
researcher_agent = Agent(
    role="Senior AI Researcher",
    goal="Discover and summarize the latest trends in AI and technology.",
    backstory="An expert in AI research who tracks emerging trends and their real-world applications.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# AI Content Strategist Agent
content_strategist_agent = Agent(
    role="Tech Content Strategist",
    goal="Transform AI research insights into compelling blog content.",
    backstory="An experienced tech writer who makes AI advancements accessible to a broad audience.",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

In [3]:
from crewai import Task

# Step 1: AI Research with Human Oversight
ai_research_task = Task(
    description=(
        "Conduct a deep analysis of AI trends in 2025. Identify key innovations, breakthroughs, and market shifts. "
        "Before finalizing, ask a human reviewer for feedback to refine the report."
    ),
    expected_output="A structured research summary covering AI advancements in 2025.",
    agent=researcher_agent,
    human_input=True  # Requires human validation before finalizing the answer
)

# Step 2: AI-Generated Blog Post with Human Review
blog_post_task = Task(
    description=(
        "Using insights from the AI Researcher, create an engaging blog post summarizing key AI advancements. "
        "Ensure the post is informative and accessible. Before finalizing, ask a human reviewer for approval."
    ),
    expected_output="A well-structured blog post about AI trends in 2025.",
    agent=content_strategist_agent,
    human_input=True  # Requires human approval before publishing
)

In [4]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[researcher_agent, content_strategist_agent],  
    tasks=[ai_research_task, blog_post_task],  
    verbose=True,  
)


In [ ]:
# Execute the workflow
result = ai_research_crew.kickoff()

# Display the final validated research output
print("\n=== Final AI Research Report ===")
print(result.raw)


In [ ]:
from IPython.display import Markdown
Markdown(result.raw)